In [9]:
import preprocessing_pipeline as pp
import mne
import numpy as np
import pickle

In [10]:
def load_raw_data(path):
    raw = pp.load_eeg_data(path, eeg_format="cnt", use_montage_creator=True, montage_type='standard_1020')
    raw = pp.add_stimulus_to_raw(raw)
    raw = pp.bandpass_filter(raw)
    raw = pp.average_rereference(raw)

    events = mne.find_events(raw, stim_channel='STI')
    return raw, events

In [11]:
def get_need_to_search_event_times(events):
    #Load in the IDs specific to word displays, as well as NeedToSearch conditional IDs
    NeedToSearchIDs = [21,22,23,24,25,26,27,28,29,30,31,32,33,4,5,6,9]

    events_times = mne.pick_events(events,include=NeedToSearchIDs)
    events_times = np.array(events_times == 4, 34, events_times) #Change 4 & 5 to 34 & 35 to allow loop to run
    events_times = np.where(events_times == 5, 35, events_times)
    events_times = np.where(events_times == 6, 36, events_times) #Change 6 & 9 to 34 & 35 to allow loop to run
    events_times = np.where(events_times == 9, 37, events_times)

    #Segment all trials into lists of events
    event_sequence_list = []
    event_sequence = []
    event_id_max = 0
    for x in events_times:
        event_info = x
        event_id = event_info[2]
        #print(event_id_max)
        if event_id > event_id_max:
            event_id_max = event_id
            event_sequence.append(event_info)
        else:
            event_sequence_list.append(event_sequence)
            event_sequence = []
            event_id_max = 0

    #Find all trials that contain both 34 and 35 e.g. NeedToSearch conditions
    NeedToSearchEventTimes = []
    CorrectEventTimes = []
    InCorrectEventTimes = []
    for x in event_sequence_list:
        event_sequence = x
        if any(34 and 35 in array for array in event_sequence) == True:
            NeedToSearchEventTimes.append(event_sequence)
        elif any(34 in array for array in event_sequence) == True:
            CorrectEventTimes.append(event_sequence)
        elif any(35 in array for array in event_sequence) == True:
            InCorrectEventTimes.append(event_sequence)

    return NeedToSearchEventTimes, CorrectEventTimes, InCorrectEventTimes



In [12]:
def get_eeg_time_segments(raw, EventTimes):

    sample_rate = 500 #TODO - get this from raw
    eeg_time_segments = []
    for event_sequence in EventTimes:
        for event in event_sequence:
            start_index =
        start_index = x[2][0]
        end_index = int(start_index + (0.8*sample_rate))
        cropped_data = raw[:, start_index:end_index]
        eeg_time_segments.append(cropped_data)
    return eeg_time_segments

SyntaxError: invalid syntax (2200075877.py, line 7)

In [ ]:
participant_number = "01"
path = fr"I:\Science\CIS-YASHMOSH\niallmcguire\dominika\Raw Data\0{participant_number}\0{participant_number}.cnt"
raw, events = load_raw_data(path)


Reading 0 ... 3507399  =      0.000 ...  7014.798 secs...


C:\Users\gxb18167\PycharmProjects\SIGIR_EEG_GAN\Development\Information-Need\Data\preprocessing_pipeline.py:137: RuntimeWarning: Could not define the number of bytes automatically. Defaulting to 2.
  raw = mne.io.read_raw_cnt(file, preload=preload)
C:\Users\gxb18167\PycharmProjects\SIGIR_EEG_GAN\Development\Information-Need\Data\preprocessing_pipeline.py:53: RuntimeWarning: Not setting positions of 2 misc channels found in montage:
['A1', 'A2']
Consider setting the channel types to be of EEG/sEEG/ECoG/DBS/fNIRS using inst.set_channel_types before calling inst.set_montage, or omit these channels when creating your montage.
  self.raw.set_montage(montage)


Creating RawArray with float64 data, n_channels=1, n_times=3507400
    Range : 0 ... 3507399 =      0.000 ...  7014.798 secs
Ready.


In [ ]:
NeedToSearchEventTimes = get_need_to_search_event_times(events)

In [ ]:
NeedToSearchEventTimes

In [177]:
ParticipantList = ["01", "02", "03", "04", "05", "06", "08", "09", "10",
                   "11", "12", "13", "14", "15", "16", "18", "19", "20",
                   "21", "22", "23", "24"]

for participant_number in ParticipantList:
    path = fr"I:\Science\CIS-YASHMOSH\niallmcguire\dominika\Raw Data\0{participant_number}\0{participant_number}.cnt"
    raw, events = load_raw_data(path)
    NeedToSearchEventTimes = get_need_to_search_event_times(events)
    NeedToSearchEEGSegments = get_eeg_time_segments(raw, NeedToSearchEventTimes)
    pickle_file_path = fr'C:\Users\gxb18167\PycharmProjects\SIGIR_EEG_GAN\Development\Information-Need\EEG-NeedToSearch-DataSegments\participant_0{participant_number}_EEG_data.pkl'

    # Open the file in binary write mode and use pickle.dump to save the data
    with open(pickle_file_path, 'wb') as pickle_file:
        pickle.dump(NeedToSearchEEGSegments, pickle_file)


Reading 0 ... 3507399  =      0.000 ...  7014.798 secs...


C:\Users\gxb18167\PycharmProjects\SIGIR_EEG_GAN\Development\Information-Need\Data\preprocessing_pipeline.py:137: RuntimeWarning: Could not define the number of bytes automatically. Defaulting to 2.
  raw = mne.io.read_raw_cnt(file, preload=preload)
C:\Users\gxb18167\PycharmProjects\SIGIR_EEG_GAN\Development\Information-Need\Data\preprocessing_pipeline.py:53: RuntimeWarning: Not setting positions of 2 misc channels found in montage:
['A1', 'A2']
Consider setting the channel types to be of EEG/sEEG/ECoG/DBS/fNIRS using inst.set_channel_types before calling inst.set_montage, or omit these channels when creating your montage.
  self.raw.set_montage(montage)


Creating RawArray with float64 data, n_channels=1, n_times=3507400
    Range : 0 ... 3507399 =      0.000 ...  7014.798 secs
Ready.
1839 events found
Event IDs: [  1   2   3   4   5   6   7   8   9  10  11  12  13  14  20  21  22  23
  24  25  26  27  28  29  30  31  32  33  34  35  36  39  44  55  56  57
  58  59  60  61  62  63  64  65  66  67  68  69  70  71  72  73  74  75
  76  77  78  79  80  81  82  83  84  85  86  87  88  89  90  91  92  93
  94  95  96  97  98  99 100 101 102 103 104 105 106 107 108 109 110 111
 112 113 114 115 116 117 118 119 120 121 122 123 124 125 126 127 128 129
 130 131 132 133 134 135 136 137 138 139 140 141 142 143 144 145 146 147
 148 149 150 151 152 154 155 156 157 158 159 160 161 162 163 164 165 166
 167 168 169 170 171 173 174 175]


In [188]:
'''
# Specify the file path where you saved the pickle file
pickle_file_path = 'your_file_path.pkl'

# Open the file in binary read mode and use pickle.load to load the data
with open(pickle_file_path, 'rb') as pickle_file:
    loaded_data = pickle.load(pickle_file)

# Now 'loaded_data' contains the data structure you saved
'''